In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PSG_PDF import PDF, report_data
from pylab import rcParams
from tqdm import tqdm

rcParams['figure.figsize'] = 12,4

import warnings
warnings.filterwarnings('ignore')

c:\Users\user\anaconda3\Lib\site-packages\fpdf\__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
day = '20240103'
store = f'processed file/{day}/report'

kine_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/kine/*csv'))]
force_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/force/*csv'))]
comment_path = f'comment_file/{day}/'

kine_name = [f"{os.path.basename(i).split('.')[0]}" for i in kine_list]
force_name = [f"{os.path.basename(i).split('.')[0]}" for i in force_list]

comment_path

'comment_file/20240103/'

In [3]:
kine_data = {}
for k, n in zip(kine_list, kine_name):
    kine_data[n] = k
    
force_data = {}
for f, n in zip(force_list, force_name):
    force_data[n] = f

In [4]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        self.cell(0, -10, 'Kookmin University Pitching Report', ln = True)

    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'imagefile/logo.png',
                160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(200, 220, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
    'PELVIS_ANGLUAR_VELOCITY_Z'        : ['PELVIS'   , 'r'],
    'TORSO_ANGLUAR_VELOCITY_Z'         : ['TORSO'    , 'g'],
    'LEAD_ELBOW_ANGULAR_VELOCITY_X'    : ['ELBOW'    , 'b'],
    'LEAD_SHOULDER_ANGULAR_VELOCITY_Z' : ['SHOULDER' , 'y'],
}

ang_cols = {
    'TORSO_PELVIS_ANGLE_Z'            : 'Hip-Shoulder Separation',
    'LEAD_ELBOW_ANGLE_X'              : 'ELBOW FLEXION',
    'LEAD_SHOULDER_ANGLE_Z'           : 'SHOULDER EXTERNAL ROTATION',          
    'LEAD_SHOULDER_ANGLE_X'           : 'SHOULDER HORIZONTAL ABDUCTION',
    'LEAD_KNEE_ANGULAR_VELOCITY_X'    : 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY',
    'LEAD_SHOULDER_ANGLE_Y'           : 'SHOULDER ABDUCTION', 
    'LEAD_KNEE_ANGLE_X'               : 'LEAD LEG KNEE FLEXION',
    'TORSO_ANGLE_X'                   : 'TRUNK FORWARD TILT',
    'TORSO_ANGLE_Y'                   : 'TRUNK LATERAL TILT',
}

ap_cols = {
    'REAR_FORCE_Y' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Y' : ['Stride Leg', 'r'],
}

vt_cols = {
    'REAR_FORCE_Z' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Z' : ['Stride Leg', 'r']
}
for name in tqdm(kine_data):
    kine = kine_data[name]
    force = force_data[name]

    k_df = pd.read_csv(kine)
    f_df = pd.read_csv(force)
    

    with open(f"{comment_path}/{name.split('_')[0]}_{name.split('_')[1]}_comment.txt", "rb") as fh:
        comment_txt = fh.read().decode('utf-8')
    
    position_kinematic = comment_txt.find('- KINEMATIC SEQUENCE & STRIDE LENGTH')
    position_Stride = comment_txt.find('- Stride')
    position_ArmCocking = comment_txt.find('- Arm Cocking')
    position_ArmAcceleration = comment_txt.find('- Arm Acceleration')
    
    kinmatic_comment = comment_txt[position_kinematic:position_Stride]
    Stride_comment = comment_txt[position_Stride:position_ArmCocking]
    ArmCocking_comment = comment_txt[position_ArmCocking:position_ArmAcceleration]
    ArmAcceleration_comment = comment_txt[position_ArmAcceleration:]
    
    k_sr = 180
    k_kh_time = k_df['kh_time'][0]
    k_fc_time = k_df['fc_time'][0] - k_kh_time
    k_fp_time = k_df['fp_time'][0] - k_kh_time
    k_mer_time = k_df['mer_time'][0] - k_kh_time
    k_br_time = k_df['br_time'][0] - k_kh_time
    stride_length = round(k_df['stride_length'][0])
    
    f_sr = 1080
    f_kh_time = f_df['kh_time'][0] 
    f_fc_time = f_df['fc_time'][0] - f_kh_time
    f_fp_time = f_df['fp_time'][0] - f_kh_time
    f_mer_time = f_df['mer_time'][0] - f_kh_time
    f_br_time = f_df['br_time'][0] - f_kh_time

    k_df = k_df.iloc[k_kh_time:int(k_br_time + k_kh_time + (k_sr * 0.2)),:].reset_index(drop=True)
    f_df = f_df.iloc[f_kh_time:int(f_br_time + f_kh_time + (f_sr * 0.2)),:].reset_index(drop=True)

    f_lead_peak_z_time = f_df['lead_peak_z'][0] - f_kh_time
    f_rear_peak_z_time = np.where(f_df['REAR_FORCE_Z'] == f_df['REAR_FORCE_Z'].max())[0][0]
    force_peak_time = round((f_lead_peak_z_time - f_rear_peak_z_time) / 1080 , 4)

    k_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)
    f_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)

    k_len = len(k_df)
    k_time = np.arange(0,k_len/k_sr, 1/k_sr)

    f_len = len(f_df)
    f_time = np.arange(0,f_len/f_sr, 1/f_sr)

    k_fc_br = k_br_time - k_fc_time
    f_fc_br = f_br_time - f_fc_time

    y   = report_data.grf(f_df, ap_cols, f_time, f_fc_time, f_mer_time, f_br_time,f_rear_peak_z_time,f_lead_peak_z_time, axis='ap')
    z   = report_data.grf(f_df, vt_cols, f_time, f_fc_time, f_mer_time, f_br_time,f_rear_peak_z_time,f_lead_peak_z_time, axis='vt')
    ang = report_data.one_angle(k_df, ang_cols, k_time, k_fc_time, k_fp_time, k_mer_time, k_br_time)
    ks  = report_data.kinematic_sequence(k_df, ks_cols, k_time, k_fc_time, k_mer_time, k_br_time)
    report_data.kinematic_sequence_comparison_bar_individual(k_df)

    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)  
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)  
    # main page
    pdf.add_page()
    pdf.set_font('helvetica', 'BU', 20)
    title = 'Kookmin University Pitching Report'
    pdf.ln(5)
    title_w = pdf.get_string_width(title) + 5
    doc_w = pdf.w
    pdf.set_x((doc_w - title_w)/2)
    pdf.cell(135, 20, title, align = 'C', ln = True)
    pdf.set_fill_color(200, 220, 255)

    # ===========================================================================================================
    # chapter
    player, trial, side, weight, height, type, ballspeed, _= name.split('_')
    if side == 'R':
        side = 'Right'
    elif side == 'L':
        side = 'Left'

    if type == 'OH':
        type = 'Overhead'
    elif type == 'UD':
        type = 'Under'
    elif type == 'TQ':
        type = 'ThreeQuater'
    elif type == 'SD':
        type = 'Side'

    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()
    ch9_link = pdf.add_link()
    ch10_link = pdf.add_link()
    ch11_link = pdf.add_link()
    ch12_link = pdf.add_link()
    ch13_link = pdf.add_link()

    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(0, 2,"",ln=1,align = '')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}   Hand : {side}    Ball Velocity : {ballspeed} km/h   Weight : {weight}kg    Height : {height} cm", align = 'C', ln = True)
    pdf.cell(0, 10,"",ln=1,align = '')
    pdf.image(f'imagefile/analysis.png',x= 30,w=150,h=55)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,7,"",ln=1,align = '')

    pdf.cell(0, 7, 'PITCHING EFFICIENCY', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. KINEMATIC SEQUENCE', ln = 1, align = '', link = ch1_link)
    pdf.cell(0,3,"",ln=1,align = '')

    pdf.cell(0, 7, 'STRIDE', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. HIP/SHOULDER SEPARATION', ln = 1, align = '', link = ch2_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. ELBOW FLEXION', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. GROUND REACTION FORCE(TRAIL LEG)', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,3,"",ln=1,align = '')

    pdf.cell(0, 7, 'ARM COCKING', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. SHOULDER EXTERNAL ROTATION', ln = 1, align = '', link = ch2_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. SHOULDER HORIZONTAL ABDUCTION', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. LEAD LEG KNEE FLEXION', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '4. LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '5. GROUND REACTION FORCE(LEAD LEG)', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')

    pdf.cell(0, 7, 'ARM ACCELERATION', ln = 1, align = '',link = ch1_link, fill = True)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '1. SHOULDER ABDUCTION', ln = 1, align = '', link = ch12_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. TRUNK FORWARD TILT', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. TRUNK LATERAL TILT', ln = 1, align = '', link = ch11_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '4. LEAD LEG KNEE FLEXION', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '5. LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '6. GROUND REACTION FORCE(LEAD LEG)', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,2,"",ln=1,align = '')

    # ===========================================================================================================
    
    # kinematic sequence

    peak_pel = round(ks['peak']['PELVIS_ANGLUAR_VELOCITY_Z']); time_pel = ks['time']['PELVIS_ANGLUAR_VELOCITY_Z']
    peak_tor = round(ks['peak']['TORSO_ANGLUAR_VELOCITY_Z']);time_tor = ks['time']['TORSO_ANGLUAR_VELOCITY_Z']
    peak_elb = round(ks['peak']['LEAD_ELBOW_ANGULAR_VELOCITY_X']);time_elb = ks['time']['LEAD_ELBOW_ANGULAR_VELOCITY_X']
    peak_sho = round(ks['peak']['LEAD_SHOULDER_ANGULAR_VELOCITY_Z']);time_sho = ks['time']['LEAD_SHOULDER_ANGULAR_VELOCITY_Z']

    total_time = k_br_time - k_fc_time
    pel_time = round(100 * (time_pel - k_fc_time) / total_time)
    tor_time = round(100 * (time_tor - k_fc_time) / total_time)
    elb_time = round(100 * (time_elb - k_fc_time) / total_time)
    sho_time = round(100 * (time_sho - k_fc_time) / total_time)
    
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_elb / peak_tor,2)
    fore_gain = round(peak_sho / peak_elb,2)

    pdf.add_page()
    pdf.set_link(ch1_link, page = 2)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'KINEMATIC SEQUENCE & STRIDE LENGTH', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    #pdf.cell(0, 0, 'The order and velocity of the pelvis, trunk and arm rotation')
    pdf.image(f'figure/kinematic.png',x=65,y=19,w=140,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/kinematic.png',x=10,y=19,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/kinematic.png',x=10,y=19,w=50,h=50)

    data_as_dict = {"Segment"                 : ["Pelvic",      "Torso",       "Elbow",     "Shoulder"],
                    "<145km/h"                  : ["649 ~ 840", "987 ~ 1174","2211 ~ 2710","4331 ~ 4884"],
                    "Peak Velocity"           : [peak_pel, peak_tor, peak_elb, peak_sho],
                    "Timing [FC - BR (100%)]" : [f"{pel_time} %", f"{tor_time} %",f"{elb_time} %",f"{sho_time} %"],
                    "Speed Gain"              : [''      , tor_gain,upper_gain,fore_gain]}
    pdf.set_y(73)

    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Kinematic Sequence',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [30,40,35,55,40],
                    x_start='C')
    pdf.set_text_color(150,0,0)
    pdf.set_y(67)
    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(5, 7, f'Stride Length : {stride_length} % (% Height)', ln = 1, align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(130)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'kinematic sequence Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(135)
    pdf.multi_cell(0, 8, kinmatic_comment, ln = True, border='True')
    
    # ===========================================================================================================

    # HIP/SHOULDER SEPARATION
    
    pdf.add_page()

    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(10)
    pdf.cell(190, 10, 'STRIDE', align = 'C', ln = True, fill = True)

    pdf.set_link(ch2_link, page = 3)
    pdf.set_y(30)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'HIP/SHOULDER SEPARATION', ln = True)
    pdf.set_y(36)
    pdf.set_font('helvetica', 'I', 10) 
    pdf.image(f'figure/Hip-Shoulder Separation.png',x=70,y=35,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/Hip-Shoulder Separation.png',x=10,y=35,w=50,h=40)
    elif side == 'Left':
        pdf.image(f'imagefile/L/Hip-Shoulder Separation.png',x=10,y=35,w=50,h=40)

    HS_fc = round(ang['fc_time']['TORSO_PELVIS_ANGLE_Z'])
    HS_max = round(ang['fc_br']['TORSO_PELVIS_ANGLE_Z'].min()) 
    HS_max_t = round(100 * (ang['fc_br_min']['TORSO_PELVIS_ANGLE_Z'] - k_fc_time) / k_fc_br)

    data_as_dict = {
                    "at Foot Contact" : [HS_fc],
                    "Min"             : [HS_max],
                    "Min Timing [FC - BR (100%)]" : [f"{HS_max_t} %"],
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip-Shoulder Separation',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    report_data.create_bullet_chart('Hip Shoulder Separation (FC)', "FC", HS_fc, -60, -39, -25, 0, unit='°')
    pdf.image(f'figure/bar//Hip Shoulder Separation (FC)_bar.png', -15, 115, 235,)

    # ===========================================================================================================

    # ELBOW FLEXION

    pdf.set_link(ch5_link, page = 3)
    pdf.set_y(160)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'ELBOW FLEXION', ln = True)
    pdf.set_y(166)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/ELBOW FLEXION.png',x=70,y=165,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/ELBOW FLEXION.png',x=10,y=165,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/ELBOW FLEXION.png',x=10,y=165,w=50,h=50)

    EF_fc = round(ang['fc_time']['LEAD_ELBOW_ANGLE_X'])
    EF_max = round(ang['fc_br']['LEAD_ELBOW_ANGLE_X'].max())  # 최댓값
    EF_max_t = round(100 * (ang['fc_br_max']['LEAD_ELBOW_ANGLE_X'] - k_fc_time) / k_fc_br)
    
    data_as_dict = {
                    "at Foot Contact" : [EF_fc],
                    "Max"        : [EF_max],
                    "Max Timing [BC - FC (100%)]" : [f"{EF_max_t} %"],
                    }
    
    pdf.set_y(220)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Elbow Flexion',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')

    report_data.create_bullet_chart('Elbow Flexion (FC)', "FC", EF_fc, 50, 76, 115, 140, unit='°')
    pdf.image(f'figure/bar//Elbow Flexion (FC)_bar.png', -15, 245, 235,)
    
    # ===========================================================================================================

    pdf.add_page()

    # Trail GRF Y

    pdf.set_link(ch11_link, page = 4)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRAIL LEG GRF (AP axis)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_ap.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Y.png',x=17,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Y.png',x=17,y=20,w=50,h=50)
    
    rear_y_fc = round(y['fc_time']['REAR_FORCE_Y'])
    max_rear_y = round(y['max']['REAR_FORCE_Y'])
    
    data_as_dict = {
                "Trail Leg" : ['at Foot Contact','Max GRF'],
                "GRF (% BW)" : [rear_y_fc, max_rear_y],
                    } 

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (AP axis)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    # ===========================================================================================================

    # Trail GRF Z

    pdf.set_link(ch12_link, page = 4)
    pdf.set_y(120)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRAIL LEG GRF (Vertical)', ln = True)
    pdf.set_y(126)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_vt.png',x=70,y=125,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Z.png',x=17,y=125,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Z.png',x=17,y=125,w=50,h=50)

    rear_z_fc = round(z['fc_time']['REAR_FORCE_Z'])
    max_rear_y = round(z['max']['REAR_FORCE_Z'])
    
    data_as_dict = {
                "Trail Leg" : ['at Foot Contact','Max GRF','Peak Time Interval'],
                "GRF (% BW)" : [rear_z_fc,max_rear_y],
                }
    
    pdf.set_y(180)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (Vertical)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    pdf.set_y(180)
    pdf.set_x(90)
    pdf.set_text_color(150,0,0)
    pdf.set_font('helvetica', 'B', 9)
    pdf.cell(5, 7, f'Blue Vertical Line ~ Red Vertical Line Time : {force_peak_time} s', ln = 1, align = '')
    pdf.set_text_color(0,0,0)

    pdf.set_y(220)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'STRIDE Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(225)
    pdf.multi_cell(0, 8, Stride_comment, ln = True, border='True')

    # ===========================================================================================================
    pdf.add_page()

    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(10)
    pdf.cell(190, 10, 'Arm cocking', align = 'C', ln = True, fill = True)

    # SHOULDER EXTERNAL ROTATION

    pdf.set_link(ch6_link, page = 5)
    pdf.set_y(30)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHOULDER EXTERNAL ROTATION', ln = True)
    pdf.set_y(36)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/SHOULDER EXTERNAL ROTATION.png',x=70,y=35,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER EXTERNAL ROTATION.png',x=10,y=35,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER EXTERNAL ROTATION.png',x=10,y=35,w=50,h=50)

    SER_fc = round(ang['fc_time']['LEAD_SHOULDER_ANGLE_Z'])
    SER_mer = round(ang['mer_time']['LEAD_SHOULDER_ANGLE_Z'])
    
    SER_max = round(ang['fc_br']['LEAD_SHOULDER_ANGLE_Z'].max())  # 최댓값
    SER_max_t = round(100 * (ang['fc_br_max']['LEAD_SHOULDER_ANGLE_Z'] - k_fc_time) / k_fc_br)

    data_as_dict = {
                    "at Foot Contact" : [SER_fc],
                    "at Max ER"       : [SER_mer],
                    "Max"             : [SER_max],
                    "Max Timing [BC - FC (100%)]" : [f"{SER_max_t} %"],
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shoulder External Rotation',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [45,45,45,45],
                    x_start='C')
    
    report_data.create_bullet_chart('Shoulder External Rotation (FC)', "FC", SER_fc, 0, 11, 48, 90, unit='°')
    pdf.image(f'figure/bar//Shoulder External Rotation (FC)_bar.png', -15, 115, 235,)

    report_data.create_bullet_chart('Shoulder External Rotation (MER)', "MER", SER_mer, 120, 163, 178, 250, unit='°')
    pdf.image(f'figure/bar//Shoulder External Rotation (MER)_bar.png', -15, 130, 235,)
    
    # ===========================================================================================================
    
    # SHOULDER HORIZONTAL ABDUCTION

    pdf.set_link(ch7_link, page = 5)
    pdf.set_y(160)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHOULDER HORIZONTAL ABDUCTION', ln = True)
    pdf.set_y(166)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/SHOULDER HORIZONTAL ABDUCTION.png',x=70,y=165,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/SHOULDER HORIZONTAL ABDUCTION.png',x=10,y=165,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/SHOULDER HORIZONTAL ABDUCTION.png',x=10,y=165,w=50,h=50)

    SHA_fc = round(ang['fc_time']['LEAD_SHOULDER_ANGLE_X'])
    SHA_mer = round(ang['mer_time']['LEAD_SHOULDER_ANGLE_X'])
    SHA_max = round(ang['fc_br']['LEAD_SHOULDER_ANGLE_X'].max())  # 최댓값
    SHA_max_t = round(100 * (ang['fc_br_max']['LEAD_SHOULDER_ANGLE_X'] - k_fc_time) / k_fc_br)
    
    data_as_dict = {
                    "at Foot Contact" : [SHA_fc],
                    "at Max ER"       : [SHA_mer],
                    "Max"        : [SHA_max],
                    "Max Timing [BC - FC (100%)]" : [f"{SHA_max_t} %"],
                    }

    pdf.set_y(220)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shoulder Horizontal Abduction',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [45,45,45,45],
                    x_start='C')
    
    report_data.create_bullet_chart('Shoulder Horizontal Abduction (FC)', "FC", SHA_fc, -100, -63, -24, 10, unit='°')
    pdf.image(f'figure/bar//Shoulder Horizontal Abduction (FC)_bar.png', -15, 245, 235,)

    report_data.create_bullet_chart('Shoulder Horizontal Abduction (MER)', "MER", SHA_mer, -50, -12, 9, 30, unit='°')
    pdf.image(f'figure/bar//Shoulder Horizontal Abduction (MER)_bar.png', -15, 260, 235,)

    # ===========================================================================================================
    pdf.add_page()

    # LEAD LEG KNEE FLEXION

    pdf.set_link(ch10_link, page = 6)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE FLEXION', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/LEAD LEG KNEE FLEXION.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE FLEXION.png',x=10,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE FLEXION.png',x=10,y=20,w=50,h=50)
    
    LKF_fc = round(ang['fc_time']['LEAD_KNEE_ANGLE_X'])
    LKF_mer = round(ang['mer_time']['LEAD_KNEE_ANGLE_X'])
    LKF_max = round(ang['fc_br']['LEAD_KNEE_ANGLE_X'].max())  # 최댓값
    LKF_max_t = round(100 * (ang['fc_br_max']['LEAD_KNEE_ANGLE_X'] - k_fc_time) / k_fc_br)
    
    data_as_dict = {
                    "at Foot Contact" : [LKF_fc],
                    "at Max ER"       : [LKF_mer],
                    "Max"        : [LKF_max],
                    "Max Timing [BC - FC (100%)]" : [f"{LKF_max_t} %"],
                    }

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Flexion',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [45,45,45,45],
                    x_start='C')
    
    report_data.create_bullet_chart('Lead Leg Knee Flexion (FC)', "FC", LKF_fc, 20, 39, 66, 100, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (FC)_bar.png', -15, 100, 235,)

    report_data.create_bullet_chart('Lead Leg Knee Flexion (MER)', "MER", LKF_mer, 0, 35, 57, 90, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (MER)_bar.png', -15, 115, 235,)

    # ===========================================================================================================

    # LEAD LEG KNEE EXTENSION ANGULAR VELOCITY

    pdf.set_link(ch9_link, page = 6)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=70,y=155,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=155,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=155,w=50,h=50)
    
    KEAV_mer = round(ang['mer_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])
    KEAV_max = round(ang['fc_br']['LEAD_KNEE_ANGULAR_VELOCITY_X'].max())  # 최댓값
    KEAV_max_t = round(100 * (ang['fc_br_max']['LEAD_KNEE_ANGULAR_VELOCITY_X'] - k_fc_time) / k_fc_br)

    data_as_dict = {
                    "at Max ER"       : [KEAV_mer],
                    "Max"        : [KEAV_max],
                    "Max Timing [BC - FC (100%)]" : [f"{KEAV_max_t} %"],
                    }

    pdf.set_y(210)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Extension Angular Velocity',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    report_data.create_bullet_chart('Lead Leg Knee Extension Angular Velocity (MAX)', "MAX", KEAV_max, 0, 218, 502, 800, unit='°/s')
    pdf.image(f'figure/bar/Lead Leg Knee Extension Angular Velocity (MAX)_bar.png', -15, 235, 235,)
    
    # ===========================================================================================================

    pdf.add_page()

    # LEAD GRF Y

    pdf.set_link(ch11_link, page = 7)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG GRF (AP axis)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_ap.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Y.png',x=17,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Y.png',x=17,y=20,w=50,h=50)
    
    lead_y_mer = round(y['mer_time']['LEAD_FORCE_Y'])
    min_lead_y = round(y['min']['LEAD_FORCE_Y'])
    
    data_as_dict = {
                "LEAD Leg" : ['at Max ER','Max GRF'],
                "GRF (% BW)" : [lead_y_mer, min_lead_y],
                    } 

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (AP axis)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    # ===========================================================================================================

    # LEAD GRF Z

    pdf.set_link(ch12_link, page = 7)
    pdf.set_y(120)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG GRF (Vertical)', ln = True)
    pdf.set_y(126)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_vt.png',x=70,y=125,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Z.png',x=17,y=125,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Z.png',x=17,y=125,w=50,h=50)

    lead_z_mer = round(z['mer_time']['LEAD_FORCE_Z'])
    max_lead_y = round(z['max']['LEAD_FORCE_Z'])
    
    data_as_dict = {
                "LEAD Leg" : ['at Max ER','Max GRF'],
                "GRF (% BW)" : [lead_z_mer,max_lead_y],
                }
    
    pdf.set_y(180)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (Vertical)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    pdf.set_y(220)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Arm cocking Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(225)
    pdf.multi_cell(0, 8, ArmCocking_comment, ln = True, border='True')

    # ===========================================================================================================
    pdf.add_page()
    
    pdf.set_font('helvetica', 'B', 15)
    pdf.set_y(10)
    pdf.cell(190, 10, 'Arm acceleration', align = 'C', ln = True, fill = True)

    # SHOULDER ABDUCTION

    pdf.set_link(ch8_link, page = 8)
    pdf.set_y(30)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHOULDER ABDUCTION', ln = True)
    pdf.set_y(36)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/SHOULDER ABDUCTION.png',x=70,y=35,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R//SHOULDER ABDUCTION.png',x=10,y=35,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L//SHOULDER ABDUCTION.png',x=10,y=35,w=50,h=50)

    SAB_br = round(ang['br_time']['LEAD_SHOULDER_ANGLE_Y'])
    SAB_max = round(ang['fc_br']['LEAD_SHOULDER_ANGLE_Y'].max())  # 최댓값
    SAB_max_t = round(100 * (ang['fc_br_max']['LEAD_SHOULDER_ANGLE_Y'] - k_fc_time) / k_fc_br)

    data_as_dict = {
                    "at Ball Release"  : [SAB_br],
                    "Max"        : [SAB_max],
                    "Max Timing [BC - FC (100%)]" : [f"{SAB_max_t} %"],
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shoulder Abduction',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    report_data.create_bullet_chart('Shoulder Abduction (BR)', "BR", SAB_br, 50, 81, 99, 120, unit='°')
    pdf.image(f'figure/bar//Shoulder Abduction (BR)_bar.png', -15, 115, 235,)

    # ===========================================================================================================
    
    # TRUNK FORWARD TILT

    pdf.set_link(ch9_link, page = 8)
    pdf.set_y(160)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK FORWARD TILT', ln = True)
    pdf.set_y(166)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/TRUNK FORWARD TILT.png',x=70,y=165,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/TRUNK FORWARD TILT.png',x=10,y=165,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/TRUNK FORWARD TILT.png',x=10,y=165,w=50,h=50)
    
    TFT_br = round(ang['br_time']['TORSO_ANGLE_X'])
    TFT_max = round(ang['fc_br']['TORSO_ANGLE_X'].max())  # 최댓값
    TFT_max_t = round(100 * (ang['fc_br_max']['TORSO_ANGLE_X'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "at Ball Release"  : [TFT_br],
                    "Max"        : [TFT_max],
                    "Max Timing [FC - BR (100%)]" : [f"{TFT_max_t} %"],
                    }

    pdf.set_y(220)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Trunk Forward Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    report_data.create_bullet_chart('Trunk Forward Tilt (BR)', "BR", TFT_br, 0, 22, 43, 100, unit='°')
    pdf.image(f'figure/bar/Trunk Forward Tilt (BR)_bar.png', -15, 245, 235,)

    # ===========================================================================================================

    # TRUNK LATERAL TILT

    pdf.add_page()
    pdf.set_link(ch10_link, page = 9)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK LATERAL TILT', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/TRUNK LATERAL TILT.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/TRUNK LATERAL TILT.png',x=10,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/TRUNK LATERAL TILT.png',x=10,y=20,w=50,h=50)
    
    TLT_br = round(ang['br_time']['TORSO_ANGLE_Y'])
    TLT_max = round(ang['fc_br']['TORSO_ANGLE_Y'].max())  # 최댓값
    TLT_max_t = round(100 * (ang['fc_br_max']['TORSO_ANGLE_Y'] - k_fp_time) / k_fc_br)

    data_as_dict = {
                    "at Ball Release"  : [TLT_br],
                    "Max"        : [TLT_max],
                    "Max Timing [FC - BR (100%)]" : [f"{TLT_max_t} %"],
                    }

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Trunk Forward Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    
    report_data.create_bullet_chart('Trunk Lateral Tilt (BR)', "BR", TLT_br, -20, 9, 23, 50, unit='°')
    pdf.image(f'figure/bar/Trunk Lateral Tilt (BR)_bar.png', -15, 100, 235,)


    # ===========================================================================================================

    # LEAD LEG KNEE FLEXION

    pdf.set_link(ch10_link, page = 9)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE FLEXION', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/LEAD LEG KNEE FLEXION.png',x=70,y=155,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE FLEXION.png',x=10,y=155,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE FLEXION.png',x=10,y=155,w=50,h=50)
    
    LKF_br = round(ang['br_time']['LEAD_KNEE_ANGLE_X'])
    LKF_max = round(ang['fc_br']['LEAD_KNEE_ANGLE_X'].max())  # 최댓값
    LKF_max_t = round(100 * (ang['fc_br_max']['LEAD_KNEE_ANGLE_X'] - k_fc_time) / k_fc_br)
    
    data_as_dict = {
                    "at Ball Release" : [LKF_br],
                    "Max"        : [LKF_max],
                    "Max Timing [BC - FC (100%)]" : [f"{LKF_max_t} %"],
                    }

    pdf.set_y(210)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Flexion',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')

    report_data.create_bullet_chart('Lead Leg Knee Flexion (BR)', "BR", LKF_br, 0, 25, 53, 90, unit='°')
    pdf.image(f'figure/bar/Lead Leg Knee Flexion (BR)_bar.png', -15, 235, 235,)

    # ===========================================================================================================
    pdf.add_page()

    # LEAD LEG KNEE EXTENSION ANGULAR VELOCITY

    pdf.set_link(ch9_link, page = 10)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.image(f'figure/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/LEAD LEG KNEE EXTENSION ANGULAR VELOCITY.png',x=10,y=20,w=50,h=50)
    
    KEAV_br = round(ang['br_time']['LEAD_KNEE_ANGULAR_VELOCITY_X'])
    KEAV_max = round(ang['fc_br']['LEAD_KNEE_ANGULAR_VELOCITY_X'].max())  # 최댓값
    KEAV_max_t = round(100 * (ang['fc_br_max']['LEAD_KNEE_ANGULAR_VELOCITY_X'] - k_fc_time) / k_fc_br)

    data_as_dict = {
                    "at Ball Release" : [KEAV_br],
                    "Max"        : [KEAV_max],
                    "Max Timing [BC - FC (100%)]" : [f"{KEAV_max_t} %"],
                    }

    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Extension Angular Velocity',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')

    # ===========================================================================================================

    # LEAD GRF Y

    pdf.set_link(ch11_link, page = 7)
    pdf.set_y(140)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG GRF (AP axis)', ln = True)
    pdf.set_y(146)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_ap.png',x=70,y=145,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Y.png',x=17,y=145,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Y.png',x=17,y=145,w=50,h=50)
    
    lead_y_br = round(y['br_time']['LEAD_FORCE_Y'])
    min_lead_y = round(y['min']['LEAD_FORCE_Y'])
    
    data_as_dict = {
                "LEAD Leg" : ['at Ball Release','Max GRF'],
                "GRF (% BW)" : [lead_y_br, min_lead_y],
                    } 

    pdf.set_y(210)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (AP axis)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    # ===========================================================================================================

    pdf.add_page()

    # LEAD GRF Z

    pdf.set_link(ch12_link, page = 7)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG GRF (Vertical)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    #pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/grf_vt.png',x=70,y=20,w=130,h=50)
    if side == 'Right':
        pdf.image(f'imagefile/R/GRF_Z.png',x=17,y=20,w=50,h=50)
    elif side == 'Left':
        pdf.image(f'imagefile/L/GRF_Z.png',x=17,y=20,w=50,h=50)

    lead_z_br = round(z['br_time']['LEAD_FORCE_Z'])
    max_lead_y = round(z['max']['LEAD_FORCE_Z'])
    
    data_as_dict = {
                "LEAD Leg" : ['at Ball Release','Max GRF'],
                "GRF (% BW)" : [lead_z_br,max_lead_y],
                }
    
    pdf.set_y(75)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (Vertical)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [94.5,94.5],
                    x_start='C')

    pdf.set_y(115)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Arm cocking Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(120)
    pdf.multi_cell(0, 8, ArmAcceleration_comment, ln = True, border='True')
    
    
    figure_file = glob('figure/*png')
    for figure in figure_file:
        os.remove(figure)
        
    pdf.output(f'{store}/{day}_{player}_{trial}_{side}_baseball_report.pdf')


100%|██████████| 9/9 [01:14<00:00,  8.32s/it]
